## Error analysis of finetune model on function name prediction

In [1]:
# Since I worked on runopod I have pretty small container storage,
# so I have to put HF stuff on network disk
import os
os.environ["HF_HOME"] = "/workspace/hf"

In [5]:
from datasets import load_dataset
# We will use only 1000 samples to get reasonable fast infernce
test_dst = load_dataset("hynky/jetbrains-community-function_name")["test"].shuffle(seed=42).select(range(1000))
# Sort by length for faster inference
test_dst = test_dst.map(lambda x: {"len": len(x["source_code"])})
test_dst = test_dst.sort("len")

In [6]:
import re
def get_fc_name_tokens(fc_name):
    """
    This function parses the function name and returns its "tokens",
    splitting on underscores and camel case.
    """
    # Split on underscores
    parts = fc_name.split('_')

    # Split camel case parts
    tokens = []
    for part in parts:
        tokens.extend(re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', part))

    return tokens


In [7]:
import numpy as np

def calculate_precision(expected, predicted):
    matching_tokens = np.sum(np.isin(predicted, expected))
    precision = matching_tokens / len(predicted) if predicted else 0
    return precision

def calculate_recall(expected, predicted):
    matching_tokens = np.sum(np.isin(predicted, expected))
    recall = matching_tokens / len(expected) if expected else 0
    return recall

def calculate_f1(precision, recall):
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall else 0
    return f1

def calculate_average_metric(expected_tokens, predicted_tokens, metric_func):
    total_metric = np.sum([metric_func(expected, predicted) for expected, predicted in zip(expected_tokens, predicted_tokens)])
    average_metric = total_metric / len(expected_tokens)
    return average_metric

In [8]:
from sklearn.metrics import f1_score, accuracy_score
import torch
from tqdm import tqdm

def predict_with_model(model, tokenizer, dataset, batch_size: int, max_length: int = 2048, max_gen_tokens=100):
    padding = True if batch_size > 1 else "do_not_pad"
    predictions = []
    for batch_start in tqdm(range(0, len(dataset), batch_size)):
        batch = dataset.select(range(batch_start, min(batch_start + batch_size, len(dataset))))
        tokenized = batch.map(lambda x: tokenizer(x["text"], padding=padding, 
                                                    add_special_tokens=False, max_length=max_length,
                                                    truncation=True, verbose=False), batched=True)
        inputs = torch.tensor(tokenized["input_ids"]).to("cuda")
        print(inputs.shape)
        attention_mask = torch.tensor(tokenized["attention_mask"]).to("cuda")
        generated_tokens = model.generate(inputs=inputs,
                                attention_mask=attention_mask,
                                max_new_tokens=max_gen_tokens)

        generated_tokens = generated_tokens.detach().to("cpu")
        predicted_names = tokenizer.batch_decode(generated_tokens[:, inputs.shape[1]:], skip_special_tokens=True)
        predictions.extend(predicted_names)

        # Free cuda memory
        del generated_tokens
        del inputs
        del attention_mask
        import gc
        gc.collect()
        torch.cuda.empty_cache()

    return predictions

def calculate_accuracy(true_tokens, predicted_tokens, tokenizer):
    # Convert tokens to text
    true_texts = [tokenizer.decode(tokens) for tokens in true_tokens]
    predicted_texts = [tokenizer.decode(tokens) for tokens in predicted_tokens]

    # Calculate accuracy
    accuracy = accuracy_score(true_texts, predicted_texts)

    return accuracy

import re

def extract_function_name(text):
    match = re.search(r"```\s*([^`]*)\s*```", text)
    if match:
        return match.group(1)
    
    match_simple = re.search(r"`\s*([^`]*)\s*`", text)
    if match_simple:
        return match_simple.group(1)
    return text

def evaluate_with_model(model, tokenizer, dataset, batch_size: int, max_length: int = 2048, max_gen_tokens=100, transform_predictions=lambda x: x):
    # model = None
    predictions = predict_with_model(model, tokenizer, dataset, batch_size, max_length, max_gen_tokens)
    predictions = transform_predictions(predictions)
    expected_tokens = [get_fc_name_tokens(x) for x in dataset["function_name"]]
    predictions_tokens = [get_fc_name_tokens(x) for x in predictions]

    precision = [calculate_precision(exp, pred) for exp,pred in zip(expected_tokens, predictions_tokens)]
    recall = [calculate_recall(exp, pred) for exp,pred in zip(expected_tokens, predictions_tokens)]
    f1 = [calculate_f1(prec, recall) for prec,recall in zip(precision, recall)]
    return {
        "accuracy": accuracy_score(predictions, dataset["function_name"]),
        "recall":  np.sum(recall) / len(predictions_tokens),
        "precision": np.sum(precision) / len(predictions_tokens),
        "f1": np.sum(f1) / len(predictions_tokens),
        "predictions": predictions
    }

## Simple prompting without finetune_model first

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
# Since we are able to fit the model in memory, we don't have to go for peft
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, use_flash_attention_2=True, device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.90s/it]


In [15]:
# Tokenize
def apply_template(x, tokenizer):
    return {"text": tokenizer.apply_chat_template(
        [{"role": "system", "content": "Generate a fitting name for the provided function. Place the suggested function name inside tripplet backtics. E.g ```adder```"},
         {"role": "user", "content": x["source_code"]}],
        tokenize=False)
    }

dataset = test_dst.map(apply_template, fn_kwargs={"tokenizer": tokenizer})
# I have to keep the batch size low because long examples would yield CUDA OOM
# Again could be better implemented by choosing batch size based on length
results_base = evaluate_with_model(model, tokenizer, dataset, 6, transform_predictions=lambda x: [extract_function_name(y) for y in x], max_length=1024)

Map: 100%|██████████| 6/6 [00:00<00:00, 1008.04 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([6, 65])


Map: 100%|██████████| 6/6 [00:00<00:00, 783.20 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([6, 66])


Map: 100%|██████████| 6/6 [00:00<00:00, 943.00 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Map: 100%|██████████| 6/6 [00:00<00:00, 397.67 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([6, 926])


Map: 100%|██████████| 4/4 [00:00<00:00, 223.23 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([4, 991])


100%|██████████| 167/167 [07:26<00:00,  2.67s/it]


In [16]:
# lets see whether names are sensible
results_base["predictions"][:10]

['incrementer',
 'sum',
 'extract',
 'increment',
 'summarizer',
 'incrementer',
 'sum',
 'getX',
 'getModule',
 'typeChooserManager']

In [12]:
# free memory
del model
import gc
gc.collect()

torch.cuda.empty_cache()

In [2]:
# I will upload the trained LoRA adapter to hub here
from peft import AutoPeftModelForCausalLM

peft_model = AutoPeftModelForCausalLM.from_pretrained("./qlora-out/")
peft_model.push_to_hub("hynky/codellama-7b-sft-lora-func-names-java-4bit")

/workspace/function_name_prediction/axolotl/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.05s/it]
CUDA extension not installed.
CUDA extension not installed.
adapter_model.safetensors: 100%|██████████| 1.28G/1.28G [00:42<00:00, 30.0MB/s] 


CommitInfo(commit_url='https://huggingface.co/hynky/codellama-7b-sft-lora-func-names-java-4bit/commit/28e94d36be189599227e59c8163c121370be0509', commit_message='Upload model', commit_description='', oid='28e94d36be189599227e59c8163c121370be0509', pr_url=None, pr_revision=None, pr_num=None)

## Finetune QLora

In [3]:
from pathlib import Path
from peft.auto import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "hynky/codellama-7b-sft-lora-func-names-java-4bit"
# Load in 4-bit as was trained
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, load_in_4bit=True, device_map="auto",
                                             use_flash_attention_2=True)

tokenizer = AutoTokenizer.from_pretrained(model.name_or_path)
tokenizer.pad_token = tokenizer.eos_token

adapter_config.json: 100%|██████████| 570/570 [00:00<00:00, 1.90MB/s]
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.60s/it]
adapter_model.safetensors: 100%|██████████| 1.28G/1.28G [00:37<00:00, 34.4MB/s]


In [9]:
def apply_template(x, tokenizer):
    return {"text": tokenizer.apply_chat_template(
        [{"role": "system", "content": f"Given the source code of a python function, suggest a fitting name for the function."},
        {"role": "user", "content": x["source_code"]}],
        tokenize=False)
    }
dataset = test_dst.map(apply_template, fn_kwargs={"tokenizer": tokenizer})
# I am not sure why the model adds space at the end of each function name
# We have abit more memory because of quantization so we can pump up the batch size a bit
results_qlora = evaluate_with_model(model, tokenizer, dataset, 8, transform_predictions=lambda preds: [pred.strip() for pred in preds])

Map: 100%|██████████| 8/8 [00:00<00:00, 1042.55 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([8, 160])


Map: 100%|██████████| 8/8 [00:00<00:00, 976.81 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([8, 144])


Map: 100%|██████████| 8/8 [00:00<00:00, 1196.83 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Map: 100%|██████████| 8/8 [00:00<00:00, 487.44 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([8, 752])


Map: 100%|██████████| 8/8 [00:00<00:00, 352.14 examples/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([8, 979])


100%|██████████| 125/125 [02:45<00:00,  1.32s/it]


In [19]:
# Table with result
# Incredible improvents
# However because I didn't spend much time with datset creation,
# there could be some sort of contamination
# of test set.
import pandas as pd

results = {
    'Base': results_base,
    'Q-LoRA': results_qlora
}

df_results = pd.DataFrame(results)
df_results


,Base,Q-LoRA
accuracy,0.072,0.396
recall,0.311549,0.642465
precision,0.287981,0.658388
f1,0.288279,0.641804
predictions,"[incrementer, sum, extract, increment, summari...","[dispose, getName, getAllFiles, dispose, getPr..."


In [20]:
# We will now continue with QLora results
# First let's see where it made mistake
import random

expected = dataset["function_name"]
predicted = results_qlora["predictions"]
errors = [i for i in range(len(expected)) if expected[i] != predicted[i]]

for i in random.sample(errors, 10):
    print(f"Example {i+1}")
    print(f"Expected: {expected[i]}")
    print(f"Predicted: {predicted[i]}")
    print(f"Source code: \n{dataset['source_code'][i]}")
    print("-"*50)
# I would say that the errors made by LLM are fairly reasonable.
# Sometimes just a parts are mistakes as can be seen in realtively high f1 score

Example 883
Expected: processError
Predicted: error
Source code: 
@Override
  public void x(final SAXParseException ex, final ValidateXmlActionHandler.ProblemType problemType) {
    if (LOG.isDebugEnabled()) {
      String error = myHandler.buildMessageString(ex);
      LOG.debug("enter: x(error='" + error + "')");
    }

    if (ApplicationManager.getApplication().isUnitTestMode()) {
      return;
    }
    ApplicationManager.getApplication().invokeLater(() -> {
        final VirtualFile file = myHandler.getProblemFile(ex);
        myErrorsView.addMessage(
            problemType == ValidateXmlActionHandler.ProblemType.WARNING ? MessageCategory.WARNING : MessageCategory.ERROR,
            new String[]{ex.getLocalizedMessage()},
            file,
            ex.getLineNumber() - 1,
            ex.getColumnNumber() - 1,
            null
        );
      }
    );
  }
--------------------------------------------------
Example 93
Expected: getBlockCommentPrefix
Predicted: getCommentStartSe

In [23]:
# Let's compare average length
# Not sure whether possible, but since I limit token generation to 100, I will not
# generate some super long function names, so that could affect the avg predicted length
avg_len_expected = sum(len(name) for name in expected) / len(expected)
avg_len_predicted = sum(len(name) for name in predicted) / len(predicted)

print(f"Average length of expected function names: {avg_len_expected}")
print(f"Average length of predicted function names: {avg_len_predicted}")


Average length of expected function names: 14.637
Average length of predicted function names: 13.972


In [24]:
# Lastly let's see how much will accuracy rise if we use stemming
# Unfornutately almost not at all only 7 sampels were fixed

from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stemmed_expected = ["_".join(stemmer.stem(token) for token in get_fc_name_tokens(name)) for name in expected]
stemmed_predicted = ["_".join(stemmer.stem(token) for token in get_fc_name_tokens(name)) for name in predicted]
errors_stemmed = [i for i in range(len(stemmed_expected)) if stemmed_expected[i] != stemmed_predicted[i]]

print(f"Accuracy after stemming: {1 - len(errors)/len(expected)}")

# Let's see which erros were fixed
fixed_indices = set(errors) - set(errors_stemmed)
print(len(fixed_indices))
for i in fixed_indices:
    print(f"Example {i+1}")
    print(f"Expected: {expected[i]}")
    print(f"Predicted: {predicted[i]}")
    print(f"Source code: \n{dataset['source_code'][i]}")
    print("-"*50)

Accuracy after stemming: 0.396
3
Example 513
Expected: setXmlSchemaVersion
Predicted: setXMLSchemaVersion
Source code: 
@Override
  public void x(XMLSchemaVersion version, @NotNull Project project) {
    getProjectResources(project).myXMLSchemaVersion = version;
    fireExternalResourceChanged();
  }
--------------------------------------------------
Example 833
Expected: initSubstitutes
Predicted: initSubstitutions
Source code: 
private boolean x() {
    if (mySubstitutions == null && myTag != null) {
      mySubstitutions = new MultiMap<>();

      XmlTag[] tags = myTag.getSubTags();

      for (XmlTag tag : tags) {
        if (equalsToSchemaName(tag, ELEMENT_TAG_NAME)) {
          final String substAttr = tag.getAttributeValue("substitutionGroup");
          if (substAttr != null) {
            String substLocalName = XmlUtil.findLocalNameByQualifiedName(substAttr);
            mySubstitutions.putValue(substLocalName, tag);
          }
        }
      }
    }
    return mySubstituti